In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import sqlite3
import re
import pandas as pd

connection = sqlite3.connect('BOOKS_DB.db')
sql = "CREATE TABLE Books (Name varchar(255) ,category varchar(255),price float, rate int)"
cursor = connection.execute(sql)

data = requests.get(f'http://books.toscrape.com/')
query = BeautifulSoup(data.text,'html.parser')
query = query.find('ul', attrs={'class':'nav nav-list'})  
query = query.find_all('a')
categories = []
for i in range( 1, len(query) ):
    categories.append(str(query[i]).split('"')[1].split('/')[3])
    

Books_headers = ['Name','category','price','rate']
Books_data = []
for category in categories:
    #print(f'category: {category} \n----------------------------')
    
    page_number = 'index.html'
    #print('First Page\n')
    while True:
        data = requests.get(f'http://books.toscrape.com/catalogue/category/books/{category}/{page_number}')
        query = BeautifulSoup(data.text,'html.parser')
    
        query_names = query.find_all('article', attrs={'class':'product_pod'}) 
        for book in query_names:
            name = str(book.find('h3')).split('"')[3]
            price = re.sub("[^0-9]", "", str(book.find('p', attrs={'class':'price_color'})).split('"')[2])
            price = float(price[:2] + '.' + price[2:])
            rate = book.find('p',attrs={'class':['star-rating One','star-rating Two','star-rating Three','star-rating Four','star-rating Five']})['class'][1]
            if rate == 'One':
                rate = 1
            elif rate == 'Two':
                rate = 2
            elif rate == 'Three':
                rate = 3
            elif rate == 'Four':
                rate = 4
            else:
                rate = 5
            Books_data.append([name,price,category,rate])
            
            sql = 'INSERT INTO Books (Name,category,price,rate) VALUES (?,?,?,?)'
            data = [name,category,price,rate]
            cursor = connection.execute(sql,data)
            
            #print(str(book_name.find('h3')).split('"')[3])
        
        query_pages = query.find_all('li', attrs={'class':'next'})
        if len(query_pages)==0:
            break

        page_number = str(query_pages).split('"')[3]
        #print('\n',page_number,'\n----------------------------')
connection.commit()
connection.close()

In [2]:
connection = sqlite3.connect('BOOKS_DB.db')
sql = "CREATE TABLE Categories (id int NOT NULL,category varchar(255) )"
cursor = connection.execute(sql)


for i in range(1,len(categories)):
    sql = 'INSERT INTO categories (id,category) VALUES (?,?)'
    data = [i-1,categories[i]]
    cursor = connection.execute(sql,data)
    
connection.commit()
connection.close()

In [3]:
with open("Books.csv" , 'w',newline="",encoding="utf-8") as Books:
    w = csv.writer(Books)
    w.writerow(Books_headers)
    w.writerows(Books_data)
Books = pd.read_csv('Books.csv')
Books

,Name,category,price,rate
0,It's Only the Himalayas,45.17,travel_2,2
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,travel_2,4
2,See America: A Celebration of Our National Par...,48.87,travel_2,3
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,travel_2,2
4,Under the Tuscan Sun,37.33,travel_2,3
...,...,...,...,...
995,Why the Right Went Wrong: Conservatism--From G...,52.65,politics_48,4
996,Equal Is Unfair: America's Misguided Fight Aga...,56.86,politics_48,1
997,Amid the Chaos,36.58,cultural_49,1
998,Dark Notes,19.19,erotica_50,5


In [4]:
#01) Write a sql query to get books that has rate >=3 and has 'Mr' in its name.
connection = sqlite3.connect('BOOKS_DB.db')

sql = "select * from Books where rate >= 3 and name LIKE 'Mr%' "
data = connection.execute(sql).fetchall()

connection.close()

data

[('Mrs. Houdini', 'historical-fiction_4', 30.25, 5)]

In [5]:
#02) Write a sql query to get books that has rate >=3 or price > 20.
connection = sqlite3.connect('BOOKS_DB.db')

sql = "select * from Books where rate >= 3 and price > 20 "
data = connection.execute(sql).fetchall()

connection.close()


In [6]:
#03) Write a sql query to get books that has rate not 3.

connection = sqlite3.connect('BOOKS_DB.db')

sql = "select * from Books where not rate = 3"
data = connection.execute(sql).fetchall()

connection.close()

In [7]:
#04) Write a sql query to get books that has price between 10 and 40 and has rate 3 or 4 or 1.
connection = sqlite3.connect('BOOKS_DB.db')

sql = "select * from Books where not rate = 3"
data = connection.execute(sql).fetchall()

connection.close()


In [8]:
#05) Write a sql query to get the top 5 most expensive books (order books desc with price column and then limit the result to the first 5).
connection = sqlite3.connect('BOOKS_DB.db')

sql = "select * from Books order by price desc limit 5 "
data = connection.execute(sql).fetchall()

connection.close()

data

[('The Perfect Play (Play by Play #1)', 'romance_8', 59.99, 3),
 ('Last One Home (New Beginnings #1)', 'fiction_10', 59.98, 3),
 ('Civilization and Its Discontents', 'psychology_26', 59.95, 2),
 ('The Barefoot Contessa Cookbook', 'food-and-drink_33', 59.92, 5),
 ('The Diary of a Young Girl', 'nonfiction_13', 59.9, 3)]

In [9]:
#06) Write a sql query to get the 3rd 10 books in the books table order first by rate desc and then by price asc.
connection = sqlite3.connect('BOOKS_DB.db')

sql = "select * from Books order by rate desc limit 20"
data = connection.execute(sql).fetchall()

connection.close()

In [ ]:
#07) Write a sql query to add a new category in categories table and add 5 books to this category in books table with title, rate and price.
connection = sqlite3.connect('BOOKS_DB.db')

sql = "select * from Books order by rate desc limit 20"
data = connection.execute(sql).fetchall()

connection.close()

In [10]:
#08) Write a sql query to update book's rate to 3 that thier price < 20£.
UPDATE books
SET rate = 3,
WHERE price < 20;

In [12]:
#09) Write a sql query to delete all books that have price > 50£ and has rate <= 2.
DELETE FROM books 
WHERE price > 50 AND rate <=2;

In [13]:
#10) Write a sql query to count the number of books that have 'Secret' in thier names and price between 10£ and 25£.
SELECT count(price)
FROM books 
WHERE name LIKE '%Secret%' AND price BETWEEN 10 AND 25

In [14]:
#11) Write a sql query to get the minimum & maximum price for all the books that have rate 5.
SELECT MIN(price),Max(price)
FROM books 
WHERE rating = 5

In [15]:
#12) Write a sql query to calculate the avg price for all the books that have rate 5.
SELECT AVG(price)
FROM books 
WHERE rating = 5

In [16]:
#13) Write a sql query to sum all book's price that have rate 2 and price between 10 and 40
SELECT sum(price)
FROM books 
WHERE rating = 2 AND price BETWEEN 10 AND 40

In [17]:
#14) Write a sql query to join both books & categories table into one new table containing book_name & category_name & book_rate and book_price.
SELECT b.name,c.category,b.rating,b.price
FROM books b
JOIN categories c
ON c.category = b.category

In [18]:
#15) Write a sql query to calculate how many books each rate has and have price between 20£ and 30£.
SELECT rating,count(rating),price
FROM books
GROUP BY rating
HAVING price BETWEEN 20 AND 30

In [19]:
#16) Write a sql query to calculate how many books each category has having count > 10.
SELECT category,count(category) as num_of_books
FROM books
GROUP BY category
HAVING num_of_books > 10

In [ ]:
#17) Write a sql query to get all books with category_name='Music' using subquery.
